# <center> "Analysis of Business Intelligence and Data Analyst related jobs in the Polish market in terms of skills required and other benefits provided" </center>



##### Author: "Hammad Ullah"
##### Date: "30.03.2023"

## Introduction
The following report shall dive deeper into the Polish market and analyze the jobs that are listed under the category of Business Intelligence and Big Data.This report is presented as a case study of a University looking to launch a new masters' program under the name of "Business Engineering" where it hopes to make sure that the course content for this program prepares the students to prosper in the corporate world and have the right skills to be able to do well in their future endeavors related to their career. Moreover, the university would like to know other details regarding the jobs offered such as average salaries based on seniority of the job as well as the language requirement as many international students who enroll in the program would be looking forward to applying for jobs in the same Polish market.


## Methodology

### Importing Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import datetime
import re
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

#### Defining User Agent

In [67]:
UserAgent = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'})

#### Gluing all the main urls together

In [205]:
no_page = list(range(1, 11))
url_pages = [f"https://nofluffjobs.com/big-data?criteria=category%3Dbusiness-intelligence,business-analyst&page={page}" for page in no_page]

### Web scraping

The very first step is the selection of a static website which I shall scrape in order to get the necessary data. The website I have chosen to analyse is as follow:

<center>  *https://nofluffjobs.com/* </center>


In the chuck of code below, we shall run the web crawler which scrapes the data from this main url pages of the sites and returns an unorganized, but content data as a data frame. This includes:

- The url of each job posted of each of the pages
- The name of the company
- The name of position

The pandas dataframe, extracted and organized, can be seen displayed below the code.


In [431]:
data_initial = {'Job Url': [],
        'Job Position': [],
        'Company Name': [],
        'Scrape Time': []}

df_url_jobs = pd.DataFrame(data_initial)

for url in url_pages:
    try:
        main_page = requests.get(url,headers=UserAgent)
        main_page.raise_for_status()
        page_content = main_page.content
        soup_obj = BeautifulSoup(page_content , "lxml")
        dom = etree.HTML(str(soup_obj))

        url_jobs_elems = dom.xpath("//div[@class='list-container ng-star-inserted']/a")
        url_jobs = [elem.get('href') for elem in url_jobs_elems]

        position_jobs_elements = dom.xpath("//h3[@data-cy = 'title position on the job offer listing']")
        position_job = [elem.text for elem in position_jobs_elements]

        comapany_name_elements = dom.xpath("//span[@data-cy = 'company name on the job offer listing']")
        company_name = [elem.text for elem in comapany_name_elements]

        scrape_time = datetime.datetime.now().strftime("%x %X")

        data_temp = [{"Job Url": url_jobs[i], "Job Position": position_job[i], "Company Name": company_name[i], "Scrape Time": scrape_time} 
                for i in range(len(url_jobs))]

        data_temp = pd.DataFrame(data_temp)
        df_url_jobs = pd.concat([df_url_jobs, data_temp], ignore_index=True)
    except Exception as e:
        #print(f"Error: {e}")
        data_temp = pd.DataFrame({'Job Url': [None], 'Job Position': [None], 'Company Name': [None], 'Scrape Time': [None]})
        df_url_jobs = pd.concat([df_url_jobs, data_temp], ignore_index=True)

df_url_jobs['Job Url'] = [f"https://nofluffjobs.com{url}?lang=en" for url in df_url_jobs['Job Url']]
df_url_jobs


,Job Url,Job Position,Company Name,Scrape Time
0,https://nofluffjobs.com/pl/job/spark-java-or-s...,Spark(Java or Scala) Developer,ComScore,03/30/23 01:12:20
1,https://nofluffjobs.com/pl/job/analityk-projek...,Analityk/Projektant systemów IT,SOFTIQ Sp. z o.o.,03/30/23 01:12:20
2,https://nofluffjobs.com/pl/job/business-analys...,Business Analyst/Scrum Master,Link Group,03/30/23 01:12:20
3,https://nofluffjobs.com/pl/job/remote-analityk...,Remote Analityk Systemowy,Connectis_,03/30/23 01:12:20
4,https://nofluffjobs.com/pl/job/analityk-biznes...,Analityk biznesowo-systemowy - Remote,Link Group,03/30/23 01:12:20
...,...,...,...,...
169,https://nofluffjobs.com/pl/job/senior-data-eng...,Senior Data Engineer,United Consult Zrt.,03/30/23 01:12:34
170,https://nofluffjobs.com/pl/job/ms-power-bi-spe...,MS Power BI Specialist (Azure),IDEMIA,03/30/23 01:12:34
171,https://nofluffjobs.com/pl/job/it-business-ana...,IT Business Analyst,Ulyssys Kft.,03/30/23 01:12:34
172,https://nofluffjobs.com/pl/job/it-business-ana...,IT Business Analyst,DPDgroup IT Solutions Hungary Kft.,03/30/23 01:12:34


### Deeper web scraping

In the chuck of code below, we shall run a web crawler which scrapes the data from the url of the each job and returns an unorganized, but content data as a data frame. This includes:

- Salary
- Category 
- Seniority
- Job Requirements 
- Other Job Details

The pandas dataframe, extracted and organized, can be seen displayed below the code.


In [433]:
data_final = {'Salary': [],
        'Category': [],
        'Seniority': [],
        'Posting Requirements': [],
        'Job Details': []}
df_main = pd.DataFrame(data_final)
for url in df_url_jobs['Job Url']:
    try:
        main_page = requests.get(url,headers=UserAgent) 
        page_content = main_page.content
        soup_obj = BeautifulSoup(page_content , "lxml")
        dom = etree.HTML(str(soup_obj))

        salary_elements = dom.xpath("(//div[@class='salary ng-star-inserted'])[1]//h4")
        salary = salary_elements[0].text

        category_elements = dom.xpath("//div[@class = 'tw-flex flex-wrap ng-star-inserted']/a")
        category = [elem.text for elem in category_elements if elem.text]
        category_text = ", ".join(category)

        seniority_elements = dom.xpath("//li[@id = 'posting-seniority']//span")
        seniority = [elem.text for elem in seniority_elements if elem.text] 
        seniority_text = ", ".join(seniority)

        posting_requirements_must_elements = dom.xpath("//div[@id = 'posting-requirements']/section[1]//span")
        posting_requirements_must = [elem.text for elem in posting_requirements_must_elements if elem.text] 
        posting_requirements_must_text = ", ".join(posting_requirements_must)

        job_details_elements = dom.xpath("//section[@id = 'posting-specs']/ul/li")
        job_details = [elem.text for elem in job_details_elements if elem.text] 
        job_details_text = ", ".join(job_details)

        data_temp = [{"Salary": salary, "Category": category_text, "Seniority": seniority_text, "Posting Requirements": posting_requirements_must_text, "Job Details": job_details_text}]

        data_temp = pd.DataFrame(data_temp)
        df_main = pd.concat([df_main, data_temp], ignore_index=True)
    except Exception as e:
        #print(f"Error: {e}")
        data_temp = pd.DataFrame({'Salary': [None], 'Category': [None], 'Seniority': [None], 'Posting Requirements': [None], 'Job Details': [None]})
        df_main = pd.concat([df_main, data_temp], ignore_index=True)

df_main

,Salary,Category,Seniority,Posting Requirements,Job Details
0,20 000 – 25 000,"Data , Spark",Mid,"Spark , Java , Scala , Linux , API , Eng...","Start ASAP , Permanent contract , Fully rem..."
1,12 000 – 19 800,Business Analysis,Mid,"BPMN , UML , SQL , Enterprise Architect , ...","Online recruitment , Start ASAP , Permanent..."
2,30 000 – 36 000,Business Analysis,Senior,"SDLC , Jira , SQL Server , English","Online recruitment , Start ASAP , Permanent..."
3,19 000 – 25 000,Business Analysis,Mid,"BPMN , UML , Polish","Online recruitment , Start ASAP , Permanent..."
4,18 000 – 22 000,Business Analysis,Mid,"REST API , UML , BPMN , Use cases , User ...","Start ASAP , Permanent contract , Fully rem..."
...,...,...,...,...,...
169,14 769 – 24 616,"Data , Python",Senior,"Python , SQL , Data engineering , ETL , A...","Online recruitment , Start ASAP , Permanent..."
170,11 077 – 12 308,Business Intelligence,Mid,"Business Intelligence , Analytics , SQL Azu...","Online recruitment , Start ASAP , Permanent..."
171,8 000 – 9 846,Business Analysis,Mid,"Business analysis , Testing , Hungarian","Start ASAP , Permanent contract , Remote fo..."
172,8 615 – 11 077,Business Analysis,Mid,"Jira , Confluence , English (B2)","Start ASAP , Permanent contract , Remote fo..."


### Combining Data Frames

Here, we combined the data frames that we got from both web scrapers in order to fulfil the prerequisite for the Data Cleaning Process

The pandas dataframe, extracted and organized, can be seen displayed below the code.


In [434]:
df_main['Job Position'] = df_url_jobs['Job Position']
df_main['Company Name'] = df_url_jobs['Company Name']
df_main = df_main.iloc[:, [6, 5, 1, 2, 0, 3, 4]]
df_main

,Company Name,Job Position,Category,Seniority,Salary,Posting Requirements,Job Details
0,ComScore,Spark(Java or Scala) Developer,"Data , Spark",Mid,20 000 – 25 000,"Spark , Java , Scala , Linux , API , Eng...","Start ASAP , Permanent contract , Fully rem..."
1,SOFTIQ Sp. z o.o.,Analityk/Projektant systemów IT,Business Analysis,Mid,12 000 – 19 800,"BPMN , UML , SQL , Enterprise Architect , ...","Online recruitment , Start ASAP , Permanent..."
2,Link Group,Business Analyst/Scrum Master,Business Analysis,Senior,30 000 – 36 000,"SDLC , Jira , SQL Server , English","Online recruitment , Start ASAP , Permanent..."
3,Connectis_,Remote Analityk Systemowy,Business Analysis,Mid,19 000 – 25 000,"BPMN , UML , Polish","Online recruitment , Start ASAP , Permanent..."
4,Link Group,Analityk biznesowo-systemowy - Remote,Business Analysis,Mid,18 000 – 22 000,"REST API , UML , BPMN , Use cases , User ...","Start ASAP , Permanent contract , Fully rem..."
...,...,...,...,...,...,...,...
169,United Consult Zrt.,Senior Data Engineer,"Data , Python",Senior,14 769 – 24 616,"Python , SQL , Data engineering , ETL , A...","Online recruitment , Start ASAP , Permanent..."
170,IDEMIA,MS Power BI Specialist (Azure),Business Intelligence,Mid,11 077 – 12 308,"Business Intelligence , Analytics , SQL Azu...","Online recruitment , Start ASAP , Permanent..."
171,Ulyssys Kft.,IT Business Analyst,Business Analysis,Mid,8 000 – 9 846,"Business analysis , Testing , Hungarian","Start ASAP , Permanent contract , Remote fo..."
172,DPDgroup IT Solutions Hungary Kft.,IT Business Analyst,Business Analysis,Mid,8 615 – 11 077,"Jira , Confluence , English (B2)","Start ASAP , Permanent contract , Remote fo..."


### Saving Data Collected as a csv

The resulting data frame is saved as a csv file below as web scraping can take a lot of time. This way, the once collected data ca be saved.

In [435]:
df_main.to_csv("FINAL_DATA_JOBS.csv", encoding='utf-8', index=False)

# Data Cleaning

The saved csv file is read and this starts off the process of data cleaning.

In [526]:
df_final = pd.read_csv('FINAL_DATA_JOBS.csv', encoding='utf-8')

#### Below, a new column is created from the 'Job Details' column which specifies where the work is Remote, Hybrid or On-Site.


In [527]:
df_final['Location'] = df_final['Job Details'].str.extract('(Remote|Fully remote|No Remote)', flags=re.IGNORECASE)
df_final['Location'] = df_final['Location'].str.replace('Hybrid', 'Remote', case=False)

#### Below, the job location and seniority such as Hybrid, Remote, Mid, Junior etc is being removed from the name of the position of the job

In [528]:
replacements = {'(REMOTE)': '',
        '(Remote)': '',
        '/': '',
        'Mid': '',
        'Senior': '',
        'Mid/Senior': '',
        'Expert': '',
        ',': '',
        'REMOTE': '',
        'Remote': '', 
        '- 100%': '',
        'Junior': ''}
df_final['Job Position'] = df_final['Job Position'].astype(str).replace(replacements, regex=True)
df_final['Job Position'] = df_final['Job Position'].astype(str).str.strip()

#### Below, the category and seniority column is cleaned by making some replacements for easy readibility purposes

In [529]:
df_final['Category'] = df_final['Category'].str.replace(',', '/', case=False)
df_final['Seniority'] = df_final['Seniority'].str.replace(',', '/', case=False)

#### Here, the salary column is being divided into new two columns which display the lower and upper limit of the salary


In [530]:
df_final[['Salary Lower Limit', 'Salary Upper Limit']] = df_final['Salary'].str.split(' – ', expand=True)

df_final['Salary Lower Limit'] = df_final['Salary Lower Limit'].str.replace(' ', '')
df_final['Salary Lower Limit'] = df_final['Salary Lower Limit'].str.replace(r'[^\x00-\x7f]', '')

df_final['Salary Upper Limit'] = df_final['Salary Upper Limit'].str.replace(' ', '')
df_final['Salary Upper Limit'] = df_final['Salary Upper Limit'].str.replace(r'[^\x00-\x7f]', '')

df_final[['Salary Lower Limit', 'Salary Upper Limit']] = df_final[['Salary Lower Limit', 'Salary Upper Limit']].apply(pd.to_numeric)
df_final.loc[df_final['Salary Upper Limit'].isna(), 'Salary Upper Limit'] = df_final.loc[df_final['Salary Upper Limit'].isna(), 'Salary Lower Limit']
df_final

,Company Name,Job Position,Category,Seniority,Salary,Posting Requirements,Job Details,Location,Salary Lower Limit,Salary Upper Limit
0,ComScore,Spark(Java or Scala) Developer,Data / Spark,Mid,20 000 – 25 000,"Spark , Java , Scala , Linux , API , Eng...","Start ASAP , Permanent contract , Fully rem...",Fully remote,20000.0,25000.0
1,SOFTIQ Sp. z o.o.,AnalitykProjektant systemów IT,Business Analysis,Mid,12 000 – 19 800,"BPMN , UML , SQL , Enterprise Architect , ...","Online recruitment , Start ASAP , Permanent...",Fully remote,12000.0,19800.0
2,Link Group,Business AnalystScrum Master,Business Analysis,Senior,30 000 – 36 000,"SDLC , Jira , SQL Server , English","Online recruitment , Start ASAP , Permanent...",Remote,30000.0,36000.0
3,Connectis_,Analityk Systemowy,Business Analysis,Mid,19 000 – 25 000,"BPMN , UML , Polish","Online recruitment , Start ASAP , Permanent...",Fully remote,19000.0,25000.0
4,Link Group,Analityk biznesowo-systemowy -,Business Analysis,Mid,18 000 – 22 000,"REST API , UML , BPMN , Use cases , User ...","Start ASAP , Permanent contract , Fully rem...",Fully remote,18000.0,22000.0
...,...,...,...,...,...,...,...,...,...,...
169,United Consult Zrt.,Data Engineer,Data / Python,Senior,14 769 – 24 616,"Python , SQL , Data engineering , ETL , A...","Online recruitment , Start ASAP , Permanent...",Remote,14769.0,24616.0
170,IDEMIA,MS Power BI Specialist (Azure),Business Intelligence,Mid,11 077 – 12 308,"Business Intelligence , Analytics , SQL Azu...","Online recruitment , Start ASAP , Permanent...",Remote,11077.0,12308.0
171,Ulyssys Kft.,IT Business Analyst,Business Analysis,Mid,8 000 – 9 846,"Business analysis , Testing , Hungarian","Start ASAP , Permanent contract , Remote fo...",Remote,8000.0,9846.0
172,DPDgroup IT Solutions Hungary Kft.,IT Business Analyst,Business Analysis,Mid,8 615 – 11 077,"Jira , Confluence , English (B2)","Start ASAP , Permanent contract , Remote fo...",Remote,8615.0,11077.0


#### Below, the Posting Requirement column is searched for whether Polish language is a necessary pre-requisite for the job or not. This is necessary due to many internationals seeking for a job.

In [533]:
df_final['Polish Requirement'] = np.where(df_final['Posting Requirements'].str.contains('Polish'), 'YES', 'NO')

#### Below, the columns of the data are arranged so that it follows some sort of a proper arrangement in terms of heirarchy of information

In [532]:
df_final = df_final.iloc[:, [0, 1, 2, 3, 10, 7, 8, 9, 5]]

#### Below, the Posting Requirements column is cleaned for easy readibility as well as redundant information has been removed

In [534]:
df_final['Posting Requirements'] = df_final['Posting Requirements'].str.replace(r'(,)*\s*\w+\s*\(\w+\)', '')
df_final['Posting Requirements'] = df_final['Posting Requirements'].str.replace(', ', ',')
df_final['Posting Requirements'] = df_final['Posting Requirements'].str.replace(' ,', ',')
df_final['Posting Requirements'] = df_final['Posting Requirements'].astype(str).str.strip()
df_final['Posting Requirements'] = df_final['Posting Requirements'].str.replace(r'\bPolish\b.*', '')
df_final['Posting Requirements'] = df_final['Posting Requirements'].str.replace(r'\bEnglish\b.*', '')
df_final['Posting Requirements'] = df_final['Posting Requirements'].apply(lambda x: x.rsplit(',', 1)[0])


## <center> RESULTING DATA FRAME  </center>

In [535]:
df_final

,Company Name,Job Position,Category,Seniority,Polish Requirement,Location,Salary Lower Limit,Salary Upper Limit,Posting Requirements
0,ComScore,Spark(Java or Scala) Developer,Data / Spark,Mid,NO,Fully remote,20000.0,25000.0,"Spark, Java, Scala, Linux, API"
1,SOFTIQ Sp. z o.o.,AnalitykProjektant systemów IT,Business Analysis,Mid,YES,Fully remote,12000.0,19800.0,"BPMN, UML, SQL, Enterprise Architect, System a..."
2,Link Group,Business AnalystScrum Master,Business Analysis,Senior,NO,Remote,30000.0,36000.0,"SDLC, Jira, SQL Server"
3,Connectis_,Analityk Systemowy,Business Analysis,Mid,YES,Fully remote,19000.0,25000.0,"BPMN, UML"
4,Link Group,Analityk biznesowo-systemowy -,Business Analysis,Mid,NO,Fully remote,18000.0,22000.0,"REST API, UML, BPMN, Use cases"
...,...,...,...,...,...,...,...,...,...
169,United Consult Zrt.,Data Engineer,Data / Python,Senior,NO,Remote,14769.0,24616.0,"Python, SQL, Data engineering, ETL"
170,IDEMIA,MS Power BI Specialist (Azure),Business Intelligence,Mid,NO,Remote,11077.0,12308.0,"Business Intelligence, Analytics"
171,Ulyssys Kft.,IT Business Analyst,Business Analysis,Mid,NO,Remote,8000.0,9846.0,"Business analysis, Testing"
172,DPDgroup IT Solutions Hungary Kft.,IT Business Analyst,Business Analysis,Mid,NO,Remote,8615.0,11077.0,Jira
